
# Facebook Data Analysis


## Getting data from Steam page with Graph Facebook API

In [92]:
#Import Libraries
import requests
import pprint
import pickle
import time
import random
import pandas as pd
import numpy as np

In [122]:
# Request Graph facebook API

token = "EAAbWYqF5NUUBADk2nUAhQAL76HMRjINdQcCFNcI1OMPCmPoZCZAvp8j2DI7uQ6B2R4ZBw7cZA5Y7VAyBz2lDQpkGgt4t4Ird1CSIAgrbnxSo97B2uYXmB0B65XSPQ7tCxw30ZAljAjqpDzdITDjpZB1skoVdSsHSkNPfXtJUdHsuxgOz4nNEsPbgrIdM0PhPgZD"

def req_facebook(req) :
    r = requests.get("https://graph.facebook.com/v2.9/" + req , {'access_token': token })
    print(r)
    return r
req = "steam/posts?fields=message%2Ccomments.limit(0).summary(true)%2Clikes.limit(0).summary(true)%2Ccreated_time"

result = req_facebook(req).json()

<Response [200]>


In [123]:
# Loop throw the results and save the next url request to data
data = [] # new list
i = 0
while True :
    try :
        time.sleep(random.randint(1,2)) # wait between 1 and 2 seconds to make the next request
        data.extend(result['data']) #put the content of the list into another list
        r = requests.get(result['paging']['next']) # next post
        result = r.json()
        i+=1
        if i> 3 :
            break;

    except: # if there's no more results 
        print("End")
        break
        

In [124]:
# Saving data into a pickle file and upload it avoiding to make the requests every time
pickle.dump(data,open("stream_data_fb.pkl","wb"))


## Load data

In [125]:
loaded_data = pickle.load(file=open("stream_data_fb.pkl","rb"))

## Setting up & Cleaning the data

In [126]:
# Pretty print of the data
#pprint.pprint(loaded_data)
loaded_data[0]

{'comments': {'data': [],
  'summary': {'can_comment': True, 'order': 'ranked', 'total_count': 0}},
 'created_time': '2017-06-13T17:10:04+0000',
 'id': '67919847338_10154732666857339',
 'likes': {'data': [],
  'summary': {'can_like': True, 'has_liked': False, 'total_count': 3}},
 'message': "Save 60% on Dying Light: The Following - Enhanced Edition during this week's Midweek Madness*!\n\n*Offer ends Friday at 10AM Pacific Time"}

In [127]:
# From nested data to a columns based data
df = pd.io.json.json_normalize(data=loaded_data)

In [131]:
df.head()

,comments.data,comments.summary.can_comment,comments.summary.order,comments.summary.total_count,created_time,id,likes.data,likes.summary.can_like,likes.summary.has_liked,likes.summary.total_count,message
0,[],True,ranked,0,2017-06-13T17:10:04+0000,67919847338_10154732666857339,[],True,False,3,Save 60% on Dying Light: The Following - Enhan...
1,[],True,ranked,0,2017-06-13T17:10:03+0000,67919847338_10154732666797339,[],True,False,3,Today's Deal: Save 50% on Tyranny !*\n\nLook f...
2,[],True,ranked,0,2017-06-13T17:10:02+0000,67919847338_10154732666727339,[],True,False,3,Save 40% on Space Hulk: Deathwing during this ...
3,[],True,ranked,18,2017-06-13T13:46:18+0000,67919847338_10154732168212339,[],True,False,163,LawBreakers is Now Available for Pre-Purchase ...
4,[],True,ranked,64,2017-06-12T14:15:37+0000,67919847338_10154729240442339,[],True,False,901,Watch the PC Gaming Show from E3 2017 live on ...


In [132]:
df.columns = df.columns.str.strip()
print(df.columns)

Index(['comments.data', 'comments.summary.can_comment',
       'comments.summary.order', 'comments.summary.total_count',
       'created_time', 'id', 'likes.data', 'likes.summary.can_like',
       'likes.summary.has_liked', 'likes.summary.total_count', 'message'],
      dtype='object')


## Drop useless columns

In [133]:
df.drop(['comments.data','comments.summary.can_comment','comments.summary.order','likes.data','likes.summary.can_like','likes.summary.has_liked'],axis=1,inplace=True)


In [134]:
df.head()

,comments.summary.total_count,created_time,id,likes.summary.total_count,message
0,0,2017-06-13T17:10:04+0000,67919847338_10154732666857339,3,Save 60% on Dying Light: The Following - Enhan...
1,0,2017-06-13T17:10:03+0000,67919847338_10154732666797339,3,Today's Deal: Save 50% on Tyranny !*\n\nLook f...
2,0,2017-06-13T17:10:02+0000,67919847338_10154732666727339,3,Save 40% on Space Hulk: Deathwing during this ...
3,18,2017-06-13T13:46:18+0000,67919847338_10154732168212339,163,LawBreakers is Now Available for Pre-Purchase ...
4,64,2017-06-12T14:15:37+0000,67919847338_10154729240442339,901,Watch the PC Gaming Show from E3 2017 live on ...


In [135]:
df.rename(columns = {'comments.summary.total_count':'total_comments'},inplace=True)
df.rename(columns = {'likes.summary.total_count':'total_likes'},inplace=True)
#df.columns = ['total_comments','id','total_likes','message']

## Cleaning null values / Checking types

In [140]:
# Fill Nan values as empty
df.fillna("",inplace=True)

In [161]:
# Check columns types
df.dtypes

total_comments     int64
created_time      object
id                object
total_likes        int64
message           object
dtype: object

In [164]:
# convert date timeto datatime
df['created_time'] = pd.to_datetime(df['created_time'])
df.iloc[0]

total_comments                                                    0
created_time                                    2017-06-13 17:10:04
id                                    67919847338_10154732666857339
total_likes                                                       3
message           Save 60% on Dying Light: The Following - Enhan...
Name: 0, dtype: object

## Data Analysis

In [167]:
# Statistics
df.describe()

,total_comments,total_likes
count,100.000000,100.000000
mean,78.780000,340.930000
std,400.999675,994.792707
min,0.000000,3.000000
25%,11.000000,117.250000
50%,26.000000,174.000000
75%,46.000000,271.250000
max,4015.000000,9908.000000


In [168]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 5 columns):
total_comments    100 non-null int64
created_time      100 non-null datetime64[ns]
id                100 non-null object
total_likes       100 non-null int64
message           100 non-null object
dtypes: datetime64[ns](1), int64(2), object(2)
memory usage: 4.0+ KB
